# Web Scraping Advanced (oDCM)

In this tutorial, we'll expand on tools like BeautifulSoup, which is essential for static websites but has limitations with dynamic content. While BeautifulSoup remains useful, Selenium allows us to handle both static and dynamic sites by simulating user actions like scrolling, clicking, and logging in. It works within a browser window, making it a more intuitive complement to BeautifulSoup, rather than a replacement.

## Learning Objectives

Students will be able to:
- Emulate user interactions on a site
  - clicking,
  - scrolling,
  - filling forms.
- Save the retrieved data as JSON files.


---
## 1. Selenium 

### 1.1 Let's recap: Why Selenium? 

In the Web Scraping 101 tutorial, we mainly used BeautifulSoup to turn HTML into a data structure that we could search and access using Python-like syntax. While it is easy to get started with this library, it has limitations when it comes to dynamic websites. That is, websites of which the content changes - even before actively refreshing it. Examples are TikTok (e.g., having an infinite scroll), or Twitch (e.g., the chat window dynamically updating).

Selenium can handle both static and dynamic websites and mimic user behavior (e.g., scrolling, clicking, logging in). It launches another web browser window in which all actions are visible which makes it feel more intuitive. 

### 1.2 Installing Selenium

<div class="alert alert-block alert-warning"><b>Installing Selenium and Chromedriver</b> 

To install Selenium and Chromedriver locally, please follow the <a href="https://tilburgsciencehub.com/topics/collect-store/data-collection/web-scraping/scrape-dynamic-websites/">Tutorial on Tilburg Science Hub</a>.
    
You can also use the code snippet below to automate the installation. Running this snippet takes a little longer each time, but the benefit is that it almost always works!
</div>



In [1]:
# Installing necessary packages
!pip3 install selenium
!pip3 install webdriver_manager

# Importing required libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# Setting up Chrome WebDriver with WebDriver Manager using Service
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Opening the 'music to scrape' website
url = "https://music-to-scrape.org/"
driver.get(url)

# Optional: Adding some wait time for the page to fully load if needed
driver.implicitly_wait(10)  # 10 seconds


If everything went smooth, your computer opened a new Chrome window, and opened `music-to-scrape`. 

__Importantly, note that you have to run the code cell always when you want to open a new instance of Chrome.__
__If you want to close Chrome, you can use `driver.quit()`.__

<div class="alert alert-block alert-info"><b>Using Google Colab</b> 

If you're using Google Colab, you don't see your browser open up manually.
    
Whenever you switch pages, just manually open that page in your browser. Although this feels like a little less interactive, you will still be able to work through this tutorial!

</div>


### 1.3 Getting Access to a Website


Next, we're going to tell the browser to visit the "Music to Scrape" website. We'll use the `driver` object we created earlier and call the `get` method, passing the URL of the website we'd like to extract data from.

In [2]:
driver.get("https://music-to-scrape.org/")

From this point, we can use BeautifulSoup as we learned previously, though we create the `res` object from the `driver` object this time. 

## 2. Using Selenium in Combination with BeautifulSoup

Selenium is powerful for interacting with dynamic websites, but once the webpage has loaded, we can still use BeautifulSoup for parsing the HTML and extracting data efficiently. BeautifulSoup is well-suited for quickly navigating and querying the page's source code, while Selenium handles the initial loading and interaction with dynamic content. This combination gives us flexibility and efficiency in our scraping tasks.

Here's an example of how to use Selenium to load a page and then pass the HTML to BeautifulSoup for parsing. We'll extract the title of the page from https://music-to-scrape.org.

In [2]:
# Initialize the Selenium WebDriver (make sure you have a WebDriver installed)
# (see code snippets above)

# Use Selenium to load the page
driver.get("https://music-to-scrape.org")

# Get the page source and pass it to BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Example: Extracting the page title using BeautifulSoup
page_title = soup.find('title').get_text()
print("Page Title:", page_title)


Page Title: We love being scraped | music-to-scrape


In this example, Selenium loads the webpage, and then we pass the page’s source code to BeautifulSoup to extract the page title.

### ✅ Exercise 1

Now, it is your turn. Use the same combination of Selenium and BeautifulSoup to store the names of the artists in the top 15 weekly tracks in a JSON dictionary.

__Hint:__

Look at the structure of the page in the browser's developer tools to find where the artist's name is located.

### ✅ Solution - Exercise 1

In [3]:
import time
# Use Selenium to load the page
driver.get("https://music-to-scrape.org")

# Wait until the website is loaded
time.sleep(2)

# Get the page source and pass it to BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Solution: Extracting the first artist's name
section = soup.find('section', attrs={'name': 'weekly_15'})

tracks = section.find_all(class_='list-group-item')

json_dic = []

for tr in tracks:
    json_dic.append({'name': tr.find('h5').get_text()})
json_dic


[{'name': 'Gabriel Yared'},
 {'name': 'Danny Williams'},
 {'name': 'Pascal Obispo'},
 {'name': 'Vangelis'},
 {'name': 'Solas'},
 {'name': 'Joi'},
 {'name': 'Stevie Ray Vaughan And Double Trouble'},
 {'name': 'Magnatune Compilation'},
 {'name': 'Mint Condition'},
 {'name': 'Enslavement Of Beauty'},
 {'name': 'Gerald Veasley'},
 {'name': 'Sofi Marinova'},
 {'name': 'Elmore James'},
 {'name': 'Snow Patrol'},
 {'name': 'Converge'}]

In this section, we combined Selenium with BeautifulSoup to scrape data from a dynamically loaded website. After using Selenium to load the page and waiting for it to fully render, we passed the HTML to BeautifulSoup for parsing. This allowed us to extract data efficiently, like the weekly top 15 tracks, and store it in a JSON-like format. This method leverages the strengths of both tools: Selenium for dynamic content handling and BeautifulSoup for data extraction.

## 3. Interacting with Dynamic Websites

### 3.1 Clicking on buttons with Selenium
In many websites, content is hidden or only appears after certain actions like clicking a button. While BeautifulSoup is excellent for parsing static HTML, it cannot perform actions such as clicking or interacting with dynamic elements. For example, a button might load new content that doesn’t appear in the initial HTML source, or content might change after interacting with certain elements on the page (e.g., loading more items, navigating through a slideshow, etc.). This is where Selenium comes in—it can mimic user interactions like clicking, scrolling, and more, making it a powerful tool for handling dynamic websites.

In this section, we’ll show you how to use Selenium to click on a button. For example, let’s try to click the “Previous Page” button on this profile page: https://music-to-scrape.org/user?username=PandaVector67.

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# Navigate to the specific user's page
driver.get("https://music-to-scrape.org/user?username=PandaVector67")

# Wait until the website is fully loaded
time.sleep(2)

# Locate and click the 'Previous Page' button by its class or identifier (depending on the site structure)
previous_button = driver.find_element(By.LINK_TEXT, "Previous Week")
previous_button.click()

# Wait a bit to allow the new page to load
time.sleep(2)

# Get the new page's source and pass it to BeautifulSoup if needed
soup = BeautifulSoup(driver.page_source, 'html.parser')

# (Optional) Print the new section title to confirm the action
print("New Section Title:", soup.find_all('h2')[1].get_text())


New Section Title: Recently Played between 2025-10-20 and 2025-10-26


__Explanation:__
1. Selenium Interaction: We used Selenium to load the webpage and then locate the “Previous Page” button.
2. Click Action: Using find_element() with the By.LINK_TEXT method, we identified the button by its label text and triggered a click.
3.  Post-Click Action: After clicking, we waited for the new content to load and optionally passed it to BeautifulSoup for further scraping.

<div class="alert alert-block alert-info"><b>Primer: Locating Elements in Selenium</b> 

<p>In Selenium, there are several ways to locate elements on a webpage. While we used <code>LINK_TEXT</code> in the example to click a button, this is just one of many methods available for finding elements. Unlike BeautifulSoup, which relies on methods like <code>find</code> and <code>find_all</code>, Selenium has its own set of strategies for interacting with elements.</p>


<p>Here are some common ways to locate elements in Selenium:</p>
<ul>
  <li><strong>By ID</strong>: <code>driver.find_element(By.ID, "element_id")</code></li>
  <li><strong>By Name</strong>: <code>driver.find_element(By.NAME, "element_name")</code></li>
  <li><strong>By Class Name</strong>: <code>driver.find_element(By.CLASS_NAME, "class_name")</code></li>
  <li><strong>By Tag Name</strong>: <code>driver.find_element(By.TAG_NAME, "tag_name")</code></li>
  <li><strong>By CSS Selector</strong>: <code>driver.find_element(By.CSS_SELECTOR, "css_selector")</code></li>
  <li><strong>By XPath</strong>: <code>driver.find_element(By.XPATH, "xpath_expression")</code></li>
</ul>

<p>It is important to note that the syntax and methods in Selenium differ from BeautifulSoup. While BeautifulSoup focuses on parsing HTML and querying the structure, Selenium is designed to interact with the browser, and thus its element locators reflect that.</p>

<p>To avoid confusion, keep in mind that the two libraries use different conventions for querying elements, and you will need to adjust your approach accordingly.</p>

<p>For more information on the various ways to locate elements in Selenium, check out the official documentation <a href="https://selenium-python.readthedocs.io/locating-elements.html" target="_blank">here</a>.</p>


</div>


## 3.2 Scrolling on a site

When scraping websites, you may encounter pages with infinite scrolling or content that only loads when you scroll down, such as social media feeds or product listings. BeautifulSoup cannot handle scrolling since it only interacts with the initial HTML source code. However, Selenium can simulate user interactions like scrolling, making it a critical tool for scraping websites with dynamic loading.

In this section, we'll cover how to scroll on a webpage using Selenium, and we'll explain different stopping rules to control the scrolling behavior.

__Example: Slow Scrolling on music-to-scrape.org__

Let's start with a simple example of scrolling through the webpage https://music-to-scrape.org. The following code snippet shows how to scroll slowly down the page using Selenium.

In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

# Navigate to the website
driver.get("https://music-to-scrape.org")

# Sleep to allow the page to load completely
time.sleep(2)

# Scroll down the page slowly
scroll_pause_time = 1
for _ in range(3):  # Scroll down 3 times
    print(f'Scrolling for the {_+1}th time')
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
    time.sleep(scroll_pause_time)  # Pause to simulate human-like scrolling


Scrolling for the 1th time
Scrolling for the 2th time
Scrolling for the 3th time


In this example:
- We load the website and use Keys.PAGE_DOWN to simulate the down arrow key, which scrolls the page down incrementally.
- We include a pause (time.sleep()) between each scroll to mimic a more natural, human-like behavior.

There are different stopping rules to control when the scrolling should end:

1. Until a certain number of iterations: You can control the scroll by a fixed number of iterations, as shown in the example above, where we scrolled 10 times. This is useful when you want to limit the depth of your scraping (see example above).
 
2. Until an element is located: You might want to stop scrolling once a specific element appears on the page, such as a "Load More" button or a particular section.
```
while not driver.find_elements(BY.CLASS_NAME, 'target-element'):
    driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
```

3. Until the end of the page is reached: To stop scrolling when the page reaches its bottom, you can compare the current scroll position to the total page height.

```
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:  # Stop if the scroll height hasn't changed
        break
    last_height = new_height
```


### ✅ Exercise 2

Now it’s your turn. Use Selenium to scroll through the https://music-to-scrape.org website and extract the name of each track displayed in 'Top 15 Weekly Tracks' after scrolling down several times.
1. Scroll down the page five times.
2. Store the names of all tracks that appear after scrolling in a JSON dictionary.

### ✅ Solution - Exercise 2

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import json

# Initialize the WebDriver
driver = webdriver.Chrome()  # Ensure you have the correct path to chromedriver
driver.get("https://music-to-scrape.org")

# Sleep to allow the page to load completely
time.sleep(2)

json_dic = []

# Scroll down the page slowly
scroll_pause_time = 1
for _ in range(5):  
    print(f'Scrolling for the {_+1}th time')
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
    time.sleep(scroll_pause_time)  # Pause to simulate human-like scrolling
    
    # Get page source and parse with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # here songs are within elements with class 'center-vertical'
    for song in soup.find_all(class_='center-vertical'):
        # Extract only the track name (it's in an 'b' tag within 'center-vertical class')
        track_title = song.find('b') 
        if track_title:
            song_name = track_title.get_text(strip=True)
            
            # Check if the extracted name is valid (non-empty) and avoid duplicates
            if song_name and song_name not in [item['name'] for item in json_dic]:
                json_dic.append({'name': song_name, 'iteration': _})



# Print or save the JSON dictionary
json_data = json.dumps(json_dic, indent=1)
print(json_data)


Scrolling for the 1th time
Scrolling for the 2th time
Scrolling for the 3th time
Scrolling for the 4th time
Scrolling for the 5th time
[
 {
  "name": "Gabriel Yared",
  "iteration": 0
 },
 {
  "name": "Danny Williams",
  "iteration": 0
 },
 {
  "name": "Pascal Obispo",
  "iteration": 0
 },
 {
  "name": "Vangelis",
  "iteration": 0
 },
 {
  "name": "Solas",
  "iteration": 0
 },
 {
  "name": "Joi",
  "iteration": 0
 },
 {
  "name": "Stevie Ray Vaughan And Double Trouble",
  "iteration": 0
 },
 {
  "name": "Magnatune Compilation",
  "iteration": 0
 },
 {
  "name": "Mint Condition",
  "iteration": 0
 },
 {
  "name": "Enslavement Of Beauty",
  "iteration": 0
 },
 {
  "name": "Gerald Veasley",
  "iteration": 0
 },
 {
  "name": "Sofi Marinova",
  "iteration": 0
 },
 {
  "name": "Elmore James",
  "iteration": 0
 },
 {
  "name": "Snow Patrol",
  "iteration": 0
 },
 {
  "name": "Converge",
  "iteration": 0
 }
]


In this solution:
- We scrolled the page five times using Keys.PAGE_DOWN.
- After scrolling, we passed the current page source to BeautifulSoup to extract the track names.
- We stored the names of all tracks found on the page, along with the iteration number, in a JSON dictionary.

### 3.3 Infinite scrolling 

So far we have specified the number of times we'd like to scroll but in some websites, infinite scrolling is a common feature in modern websites (e.g. Twitter). In such sites, new content is dynamically loaded as the user scrolls down the page. However, it poses a unique challenge for web scraping because the HTML content changes dynamically as new data is loaded. Therefore, to effectively scrape data from such websites, we need to scroll down the page, wait for new content to load, and then capture the updated HTML. 



__Example: Infinite Scrolling on music-to-scrape.org__

In this example, we will retrieve all the recently played track titles for the user "Pixel25" from the following page: [https://music-to-scrape.org/user?username=Pixel25](https://music-to-scrape.org/user?username=Pixel25).

(Note: While music-to-scrape.org loads all its data at once and isn't fully dynamic, this code serves as a demonstration of how to handle infinite scrolling on websites that do load content dynamically.)

### ✅ Exercise 3

Please run the cell below.

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import json

# Initialize the WebDriver
driver = webdriver.Chrome()
driver.get("https://music-to-scrape.org/user?username=Pixel25")

# Allow the page to load
time.sleep(2)

track_titles = []

# Infinite scrolling to the bottom
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Pause for a moment to allow content to load
    time.sleep(1)

    # Get the current scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    # Break the loop if the scroll height has not changed
    if new_height == last_height:
        break
    last_height = new_height

# Get the page source and parse it with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Find the table containing track information
table = soup.find('table', class_='table-striped')
if table:
    # Find all rows in the table body
    rows = table.find('tbody').find_all('tr')
    for row in rows:
        # Extract text from the first <td> element for the track title
        columns = row.find_all('td')
        if columns:
            track_title = columns[0].get_text(strip=True)
            if track_title and track_title not in track_titles:
                track_titles.append(track_title)

# Print the titles in JSON format
song_data = json.dumps(track_titles, indent=2)
print(song_data)



The chromedriver version (127.0.6533.119) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (128.0.6613.138); currently, chromedriver 128.0.6613.137 is recommended for chrome 128.*, so it is advised to delete the driver in PATH and retry


[
  "Ghost of the Ocean (Live)",
  "Tables Turn",
  "Your Heart Can Sing",
  "Con La Tinta De Mi Sangre",
  "God Is The Answer (Footprints In The Sand Album Version)",
  "Lunes Por La Madrugada",
  "The Imperial March from The Empire Strikes Back",
  "Sapphire",
  "From A Jack To A King",
  "Irish Medley - Jigs",
  "The World'll Be OK",
  "That's A Woman",
  "The drifter",
  "Mangos",
  "Don't Owe Me Nothin'",
  "Anvi Viv?",
  "Shaggy Dog Songs",
  "Beauty Supply And Demand",
  "Pu**y Pop",
  "Christmas Time"
]


### ✅ Solution - Exercise 3

- We used a `while` loop to simulate infinite scrolling:
  - Scrolled to the bottom of the page using JavaScript (`window.scrollTo`).
  - Paused for a moment (`time.sleep(1)`) to allow content to load.
  - Checked the page height to determine if new content was loaded; broke out of the loop when no more content was found.
- After completing the scrolling, passed the page source to BeautifulSoup for HTML parsing.
- Located the table containing track information using its class (`'table-striped'`).
- Iterated through each row in the table, extracting the text from the first column (`<td>`) of each row, which contains the track titles.
- Added each unique track title to the `track_titles` list.
- Converted the list of track titles to a JSON format and printed the results.

# 3.4 Filling in forms

Filling in forms is a common task when interacting with websites, especially when you need to perform actions like logging in, searching, or submitting data. While BeautifulSoup is excellent for parsing static HTML, it cannot interact with forms or simulate user inputs. Selenium, on the other hand, can fill in forms, submit them, and interact with various input fields on a page, making it ideal for tasks like automated searches.

In this section, we’ll explore how to use Selenium to fill in a search form, query for artist names, and extract results. We'll query the search bar on https://music-to-scrape.org using artist names found on the page itself.

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# Navigate to the website
driver.get("https://music-to-scrape.org")
time.sleep(2)

artists = ['Adrenalin', 'Presets', 'Lupe']

# Loop through each artist and perform a search
for artist_name in artists:
    # Locate the search input field
    search_bar = driver.find_element(By.NAME, 'query')

    # Clear any previous input, then type the artist's name and submit the form
    search_bar.clear()
    search_bar.send_keys(artist_name)
    search_bar.send_keys(Keys.RETURN)

    # Wait for search results to load
    time.sleep(2)


In this example, we combine Selenium and BeautifulSoup to automate searches on the website music-to-scrape.org, search for multiple artist names, and extract results for each query. Here's a breakdown of how the process works:

1. __Loading the Website:__     We start by navigating to the website using Selenium's driver.get() method and give the page a moment to load using time.sleep().
2. __Looping Through Artists:__    We define a list of artist names (['Adrenalin', 'Presets', 'Lupe']) and loop through each name to perform a search on the site.
3. __Interacting with the Search Bar:__  For each artist, we locate the search input field using driver.find_element(By.NAME, 'query'). We clear the field, input the artist's name, and simulate pressing the return key to submit the search.
4. __Waiting for Results:__     After submitting the search, we wait for the results to load using another time.sleep(), allowing the page to update with the new search results.

### ✅ Exercise 4

Extend the code snippet above to store all search results in a JSON dictionary (stored on disk, new-line separated).

### ✅ Solution - Exercise 4

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import json

# Navigate to the website
driver.get("https://music-to-scrape.org")
time.sleep(1)

artists = ['Adrenalin', 'Presets', 'Lupe']

f = open('output.json', 'a')

# Loop through each artist and perform a search
for artist_name in artists:
    # Locate the search input field
    search_bar = driver.find_element(By.NAME, 'query')

    # Clear any previous input, then type the artist's name and submit the form
    search_bar.clear()
    search_bar.send_keys(artist_name)
    search_bar.send_keys(Keys.RETURN)

    # Wait for search results to load
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    results = soup.find(class_='container').find_all(class_='list-group-item')
    counter = 0
    for row in results:
        counter = counter + 1
        res = {} # empty dictionary
        res['query'] = artist_name
        res['rank'] = counter # rank of search result
        res['type'] = row.find(class_='center-text').find('p').get_text()
        res['result'] = row.find(class_='center-text').get_text().strip()
        f.write(json.dumps(res))
        f.write('\n')
f.close()
        


In this exercise, we’ve walked through how to automate the process of filling in a search form, extracting the resulting data, and saving it in a structured format using Selenium and BeautifulSoup.

Here's a quick summary:
1. After submitting each search, we pause briefly using `time.sleep()` to allow the page to load fully before parsing the content. This ensures that we capture the updated results from the search.
2. Once the results page has loaded, we use BeautifulSoup to parse the page’s HTML. We specifically look for the `container` section where the search results are displayed, with each result being located within a `list-group-item` class. This allows us to accurately extract the data we’re interested in.
3. For each result, we create a JSON-like dictionary to store:
- **The artist name** (`query`) that was used in the search.
- **The rank** (`rank`) of each result, incremented for each item in the list.
- **The type of result** (`type`), such as "Song" or "Album", which we find in the `<p>` tag within each result.
- **The full result text** (`result`), which includes the specific details about the result, extracted from the relevant element.
4. We then write each result into an `output.json` file, ensuring each search result is stored as an individual JSON object. This allows us to save our scraped data in a structured and reusable format for future analysis.


# 4. Summary

In this tutorial, you've explored how to combine Selenium and BeautifulSoup to scrape data from dynamic websites effectively. Here's a quick summary of key takeaways:

- Selenium vs. BeautifulSoup:
  - Selenium: Ideal for interacting with dynamic websites, such as handling forms, clicking buttons, and simulating user actions like scrolling. It works by controlling a browser and mimicking user behavior.
  - BeautifulSoup: Best suited for parsing static HTML content and extracting data from the page source. It cannot interact with dynamic elements but is excellent for navigating and querying structured HTML.
-   Using Them Together: By combining Selenium for interaction and BeautifulSoup for parsing, you can scrape dynamic sites, automate form submissions, extract data, and store it efficiently in formats like JSON.

This combination allows you to tackle more complex scraping tasks by automating the interaction with dynamic content while still benefiting from BeautifulSoup’s simplicity and efficiency in data extraction.